In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = 16,8

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from scipy.sparse import coo_matrix,csr_matrix

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# # colab
# #플레이 리스트
# train = pd.read_json('/content/drive/MyDrive/프로젝트/data/train.json')
# train['tag__'] = train['tags'].apply(lambda x: (" ").join(x))

# #곡 정보
# song_meta = pd.read_json('/content/drive/MyDrive/프로젝트/data/song_meta.json')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#플레이 리스트
train = pd.read_json('data/train.json')
train['tag__'] = train['tags'].apply(lambda x: (" ").join(x))


#곡 정보
song_meta = pd.read_json('data/song_meta.json')
song_meta['artist_name_basket__'] = song_meta['artist_name_basket'].apply(lambda x: (" ").join(x))
song_meta['song_gn_gnr_basket__'] = song_meta['song_gn_gnr_basket'].apply(lambda x: (" ").join(x))

In [ ]:
# 장르 
genre = pd.read_json('data/genre_gn_all.json', typ='series')
genre = pd.DataFrame(genre)
genre.reset_index(inplace=True)
genre.columns = ['gnr_code', 'gnr_name']
genre.head()

,gnr_code,gnr_name
0,GN0100,발라드
1,GN0101,세부장르전체
2,GN0102,'80
3,GN0103,'90
4,GN0104,'00


In [ ]:
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artist_name_basket__
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,Various Artists
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,Murray Perahia
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,Peter Gabriel
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,Matoma
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,Jude Law
...,...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984,Fela Kuti
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985,Cyndi Lauper
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986,윤종신
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987,Nature Piano


In [ ]:
song_meta[song_meta['artist_name_basket__'] == "Various Artists"]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artist_name_basket__
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,Various Artists
15,"[GN1601, GN1613]",20140915,매장 음악으로 좋은 상큼한 클래식 피아노 (매장 음악),186334,[2727],Faure: Romance Sans Paroles In A Flat Major Op...,[GN1600],[Various Artists],15,Various Artists
26,[GN0901],20100924,2010 CF 배경음악 Vol. 1,1030176,[2727],Move It Like This (Remake Ver.),[GN0900],[Various Artists],26,Various Artists
45,"[GN2001, GN2008]",20071212,Silverport Best,362126,[2727],La Bomba & Joel (Tropicalisimo Mix),[GN2000],[Various Artists],45,Various Artists
53,"[GN2102, GN2101]",20080530,항상 기쁨 주는 찬양,382576,[2727],내가 그리스도와 함께,[GN2100],[Various Artists],53,Various Artists
...,...,...,...,...,...,...,...,...,...,...
707882,"[GN2002, GN2001]",20011100,Chanson Chanson,10355,[2727],"Parole, Parole",[GN2000],[Various Artists],707882,Various Artists
707902,"[GN0703, GN0701]",20120202,신바람 트로트 디스코왕,2074708,[2727],토요일밤은 밤이 좋아,[GN0700],[Various Artists],707902,Various Artists
707908,"[GN1601, GN1602, GN1614, GN1613]",20150610,잠재된 내면의 세계로.Vol.2,2323148,[2727],Weber : Sonata No.1 In F Major - III. - Rondo ...,[GN1600],[Various Artists],707908,Various Artists
707972,[GN0901],20150728,"추억의 DJ들이 추천하는 7080 롤라장 댄스 베스트 (빌보드 빅히트 팝송댄스, 유...",2332003,[2727],Step By Step,[GN0900],[Various Artists],707972,Various Artists


In [ ]:
train
# 감정분석을

,tags,id,plylst_title,songs,like_cnt,updt_date,tag__
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,락
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,추억 회상
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,까페 잔잔한
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,연말 눈오는날 캐럴 분위기 따듯한 크리스마스캐럴 겨울노래 크리스마스 겨울왕국 크리스마스송
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,댄스
...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,록메탈 밴드사운드 록 락메탈 메탈 락 extreme
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,일렉
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,담시 가족 눈물 그리움 주인공 나의_이야기 사랑 친구
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,잔잔한 버스 퇴근버스 Pop 풍경 퇴근길


---
---
---

# 태그 유사도

In [ ]:
# 플레이리스트의 모든 곡을 하나로 합치고 set으로 중복을 제거한다.
all_tags = set(np.concatenate(train['tags']))

#중복을 제외한 플레이리스트 내 등장한 전체 곡 수
n_tags = len(all_tags)

#새 인덱스와 기존 인덱스간 인코딩 디코딩을 위한 딕셔너리 초기화.
# song_to_newid[원본인덱스] -> 바뀐 인덱스
# newid_to_song[바뀐인덱스] -> 원본인덱스
tag_to_newid  = dict()
newid_to_tag = dict()

In [ ]:
# 위 딕셔너리를 채운다.
for i, t in enumerate(all_tags):
    tag_to_newid[t] = i
    newid_to_tag[i] = t

In [ ]:
#기존 곡 id를 새 id로 바꾼 컬럼을 만든다.
train['tag_newid'] = train['tags'].apply(lambda x: [tag_to_newid[tag] for tag in x])
display(train['tag_newid'])

0                                                    [4161]
1                                            [26378, 22792]
2                                             [6101, 11326]
3         [17723, 11790, 2701, 17136, 26948, 20339, 1298...
4                                                   [13681]
                                ...                        
115066      [27926, 27691, 13307, 23548, 8028, 4161, 21184]
115067                                              [21167]
115068    [5226, 28065, 23780, 1166, 26273, 18198, 4910,...
115069              [11326, 15067, 2447, 28580, 7075, 1070]
115070                           [22565, 14450, 9615, 7522]
Name: tag_newid, Length: 115071, dtype: object

In [ ]:
# 각 플레이리스트 별 곡 수가 필요함
train['tag_cnt'] = train['tags'].map(lambda x : len(x))

#각 플레이리스트 id를 각 플레이리스트 곡 수 만큼 반복
row = np.repeat(range(len(train)), train['tag_cnt'])
row.shape

(476331,)

In [ ]:
#각 플레이 리스트에 있는 곡들을 전부 붙이면 된다.
col =  np.concatenate(train['tag_newid'])
col.shape

(476331,)

In [ ]:
# 우리는 평점에 대한 정보가 없으므로 data는 전부 1인 array가 된다.
data = np.ones(col.shape[0])

In [ ]:
# 위 데이터로 희소행렬을 만든다.
ply_tag_table = csr_matrix((data, (row,col)), shape= (len(train), n_tags))

In [ ]:
#행이 플레이리스트, 열이 곡인 행렬
ply_tag_table

<115071x29160 sparse matrix of type '<class 'numpy.float64'>'
	with 476331 stored elements in Compressed Sparse Row format>

In [ ]:
#곡 간 유사도 계산을 위해 곡이 행으로 플레이리스트가 열로 와야한다.
tag_ply_table = ply_tag_table.transpose()

#코사인 유사도 계산.
tag_sim = cosine_similarity(tag_ply_table, tag_ply_table, dense_output = False)
#곡 수 x 곡 수 크기의 유사도 행렬이 만들어짐.

In [ ]:
tag_sim

<29160x29160 sparse matrix of type '<class 'numpy.float64'>'
	with 769622 stored elements in Compressed Sparse Row format>

In [ ]:
tag_df =pd.DataFrame(tag_sim.toarray(), 
             columns = newid_to_tag.values(),
             index = newid_to_tag.values())

In [ ]:
def tag_recomm(tag_name,top_n = 30):
    top = tag_df[[tag_name]].sort_values(tag_name,ascending=False)[:top_n]
    top.columns = ['similarity']
    
    return top

In [ ]:
tag_recomm('아이유')

,pred
아이유,1.000000
IU,0.294948
효리네민박,0.268918
이상순,0.252578
이효리,0.224422
아이유콘서트,0.218739
이지은,0.195646
JTBC,0.175625
아이유노래모음,0.171191
dlwlrma,0.169435


---
---
---
# plylst_title

In [ ]:
import nltk
import pandas as pd
from konlpy.tag import Okt
from sklearn import linear_model

In [ ]:
train = pd.read_json('data/train.json')
train

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


In [ ]:
# 불용어 불어와서 리스트 형태로 바꿔주기
stopwords = pd.read_csv('data/stopword.csv')
stopwords_list = stopwords.iloc[:,0].tolist()
stopwords_list

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를',
 '들면',
 '들자면',
 '저',
 '소인',
 '소생',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수',
 '없다',
 '해서는',
 '안된다',
 '뿐만',
 '아니라',
 '만이',
 '아니다',
 '만은',
 '막론하고',
 '관계없이',
 '그치지',
 '않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지',
 '따지지',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만',
 '못하다',
 '하는',
 '편이',
 '낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이',
 '외에',
 '밖에',
 '하여야',
 '비로소',
 '한다면',
 '몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할',
 '생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게',
 '함으로써',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에',
 '틀림없다',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간'

In [ ]:
#텍스트 정제 (특수기호 제거)
import re

txt = []

for i, document in enumerate(train['plylst_title']):
    document = re.sub(r'[^ A-Za-z0-9가-힣]', '', document) #특수기호 제거, 정규 표현식    
    txt.append(document.strip()) # 문장 왼쪽 오른쪽에 있는 공백 제거

In [ ]:
txt

['여행같은 음악',
 '요즘 너 말야',
 '편하게 잔잔하게 들을 수 있는 곡',
 '크리스마스 분위기에 흠뻑 취하고 싶을때',
 '추억의 노래',
 '2017 Pop Trend',
 '짝사랑고백사랑이별슬픔 감성을 자극하는곡들',
 '멍청이 내맘도 몰라',
 'DANCING IN THE MOONLIGHT 01',
 '록메탈 Written by 이일우',
 '걸그룹 땐쓰쏭',
 '노래로 의지를 불태우자 1일1다짐 st용 프로필뮤직',
 '지친 너를 위로해줄 제목편지 준비해봤어',
 '트렌디하고 그루브한 힙합알앤비 MUSIC',
 'Autumn in Jazz',
 '걍게임할때듣는음악',
 '나만 알고싶은 노래들',
 '축가듀엣',
 '이렇게 비 내리는 날이면 너도 내 생각 할까',
 '올림픽 스밍 목록',
 '조용히 맥주 한잔 할때새벽감성',
 '',
 '추억의 명화를 만나는 시간 영화 OST 모음',
 '기분좋은 여름날',
 '지친 하루 끝 힐링이 필요한 당신에게 추천하는 인디곡',
 '새벽 찬바람이 침대위를 감쌀때국외',
 '기분좋은 햇살같은 숨겨진 명곡들',
 '사랑스러운 남녀듀엣',
 '즐겨듣는소래',
 '닥힙추 8월 Playlist',
 '거슈윈에서 스타워즈까지 미국 대표 클래식',
 '겨울이 기다리며 미리 들어요',
 '아침이 상쾌해지는 빅밴드의 스윙스윙',
 '여행가는 길 이런음악과 함께 고고씽',
 '믿고듣는 하우스 일렉트로니카 POP 2',
 '프로포즈 음악',
 '보아일본곡',
 '안다르 레깅스 입고 운동 시작하자',
 '발라드로 감성을 발라드림슬픔주의',
 '크리스마스를 책임질 캐롤 모음',
 '아무것도 위로 되지 못할 때 위로가 되어준 음악',
 '트렌디한 Cafe Pop Music',
 '890년대 우리나라 라디오에서 흘러나오던 명곡들',
 '영화 Jazz를 만나다 294',
 '우울하고 시크하',
 '극도로 우울할때',
 '폭팔적인 밴드 ROCK 모음집',
 '금쪽같은 휴일 혼자만의 시간을 방해하지 말아요 2',
 'Electronica